In [1]:
###################################### FILTRADO DE MONEDAS PARA ESTRATEGIA SANTAAAAAA
###################################################################################################################
import modulos as md
import numpy as np
lista=['XVGUSDT']
lista=md.lista_de_monedas()
for symbol in lista:
    try:
        data = md.estrategia_santa(symbol, tp_flag = False)
        resultado = md.backtesting(data, plot_flag = False)
        if resultado['Profit Factor']>1 or resultado[6] >= 1:
        #if resultado[17]>0:
            print(f"{symbol} - retorno: {md.truncate(resultado[6],2)}% - trades: {resultado[17]} - Profit Factor: {resultado['Profit Factor']}")
    except Exception as falla:
        pass

C:\Users\Luis Gomez Freites\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


EOSUSDT - retorno: 7.04% - trades: 9 - Profit Factor: 3.019326425427629
XLMUSDT - retorno: 0.07% - trades: 3 - Profit Factor: 2.3752330599136346
ADAUSDT - retorno: 0.52% - trades: 3 - Profit Factor: 22.261324239549523
ONTUSDT - retorno: 1.4% - trades: 9 - Profit Factor: 3.0490280382006505
BATUSDT - retorno: 0.21% - trades: 6 - Profit Factor: 2.312095713089259
VETUSDT - retorno: 0.0% - trades: 4 - Profit Factor: 1.4390443699482742
THETAUSDT - retorno: 5.72% - trades: 6 - Profit Factor: 18.156705779613535
KNCUSDT - retorno: 0.33% - trades: 3 - Profit Factor: 4.842732323884824
ZRXUSDT - retorno: 0.62% - trades: 8 - Profit Factor: 1.8353052294471581
OMGUSDT - retorno: 8.8% - trades: 12 - Profit Factor: 3.38109959352942
SXPUSDT - retorno: 2.04% - trades: 12 - Profit Factor: 1.6322992572603674
KAVAUSDT - retorno: 4.0% - trades: 3 - Profit Factor: 10.888840858306583


c:\LUCHO\personal\repopersonal\luchocrypto\FENIX\modulos.py:313: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data, Fenix, cash=1000)
c:\LUCHO\personal\repopersonal\luchocrypto\FENIX\modulos.py:313: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data, Fenix, cash=1000)


CRVUSDT - retorno: 2.79% - trades: 2 - Profit Factor: 21.42167182662497
SUSHIUSDT - retorno: 2.26% - trades: 4 - Profit Factor: 7.121637183688029
SRMUSDT - retorno: 2.15% - trades: 29 - Profit Factor: 1.2337344898005338
ICXUSDT - retorno: 0.93% - trades: 7 - Profit Factor: 3.509649852791322
STORJUSDT - retorno: 13.41% - trades: 23 - Profit Factor: 2.2304980741493874
BLZUSDT - retorno: 0.13% - trades: 10 - Profit Factor: 1.658257532192405
FLMUSDT - retorno: 0.44% - trades: 10 - Profit Factor: 2.8552210150021846
TOMOUSDT - retorno: 13.4% - trades: 6 - Profit Factor: 7.799918675423329
RENUSDT - retorno: 0.06% - trades: 5 - Profit Factor: 2.73467695661623
MATICUSDT - retorno: 3.92% - trades: 5 - Profit Factor: nan
BELUSDT - retorno: 2.31% - trades: 12 - Profit Factor: 1.3914755480001564
ALPHAUSDT - retorno: 0.17% - trades: 3 - Profit Factor: 8.794467603748465
SKLUSDT - retorno: 0.02% - trades: 7 - Profit Factor: 1.4624078668495544
GRTUSDT - retorno: 1.32% - trades: 12 - Profit Factor: 1.99

In [1]:
###################################### FILTRADO DE MONEDAS PARA ESTRATEGIA BB
###################################################################################################################

import modulos as md
import numpy as np
lista=['XTZUSDT']
lista=md.lista_de_monedas()
for symbol in lista:
    try:
        data = md.estrategia_bb(symbol, tp_flag = True)
        resultado = md.backtesting(data, plot_flag = False)
        if resultado['Profit Factor'] > 1 and resultado[6]/resultado[17] >=0.3:
            print(f"{symbol} - retorno: {md.truncate(resultado[6],2)}% - trades: {resultado[17]} - Profit Factor: {md.truncate(resultado['Profit Factor'],2)}")
    except Exception as falla:
        pass

C:\Users\Luis Gomez Freites\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


BATUSDT - retorno: 2.28% - trades: 7 - Profit Factor: 4.08


c:\LUCHO\personal\repopersonal\luchocrypto\FENIX\modulos.py:313: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data, Fenix, cash=1000)
c:\LUCHO\personal\repopersonal\luchocrypto\FENIX\modulos.py:313: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data, Fenix, cash=1000)


RUNEUSDT - retorno: 4.84% - trades: 6 - Profit Factor: 2.86
SRMUSDT - retorno: 2.68% - trades: 5 - Profit Factor: 1.9
FTMUSDT - retorno: 4.89% - trades: 8 - Profit Factor: 3.6
ENJUSDT - retorno: 3.57% - trades: 7 - Profit Factor: 5.46
OCEANUSDT - retorno: 4.5% - trades: 11 - Profit Factor: 2.91
BELUSDT - retorno: 4.65% - trades: 6 - Profit Factor: 2.74
CTKUSDT - retorno: 5.21% - trades: 9 - Profit Factor: 3.09
MINAUSDT - retorno: 8.56% - trades: 11 - Profit Factor: 4.12
PHBUSDT - retorno: 3.14% - trades: 8 - Profit Factor: 1.61
SUIUSDT - retorno: 4.35% - trades: 11 - Profit Factor: 1.64
COMBOUSDT - retorno: 5.01% - trades: 8 - Profit Factor: 1.96
